<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## RNN one-to-one

#### Datos
El objecto es utilizar una serie de sucuencias númericas (datos sintéticos) para poner a prueba el uso de las redes RNN. Este ejemplo se inspiró en otro artículo, lo tienen como referencia en el siguiente link:\
[LINK](https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras/)

In [ ]:
import re

import numpy as np
import pandas as pd

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, SimpleRNN
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

In [ ]:
# Generar datos sintéticos
X = list()
y = list()
X = [x+1 for x in range(20)]

# "y" (target) se obtiene como cada dato de entrada multiplicado por 15
y = [x * 15 for x in X]

print("datos X:", X)
print("datos y:", y)

In [ ]:
# Cada dato X lo transformarmos en una matriz de 1 fila 1 columna (1x1)
X = np.array(X).reshape(len(X), 1, 1)
print("datos X:", X)

In [ ]:
y = np.asanyarray(y)
y.shape

### 2 - Entrenar el modelo (RNN y LSTM)

In [ ]:
input_shape = X[0].shape
input_shape

In [ ]:
output_shape = 1

In [ ]:
# Comenzamos con una RNN clásica
# En general una celda RNN clásica ya no se utiliza, es solo a modo de ejemplo
model = Sequential()
model.add(SimpleRNN(64, activation='relu', input_shape=input_shape))
model.add(Dense(output_shape))
model.compile(loss='mse',
              optimizer="Adam")

model.summary()

In [ ]:
hist = model.fit(X, y, epochs=500, validation_split=0.2, batch_size=5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist.history['loss']) + 1)
sns.lineplot(x=epoch_count,  y=hist.history['loss'], label='train')
sns.lineplot(x=epoch_count,  y=hist.history['val_loss'], label='valid')
plt.show()

In [ ]:
# Ensayo
# x = 30
# y_test = x * 15

x_test = 30
y_test = x_test * 15
test_input = np.array([x_test])
test_input = test_input.reshape((1, 1, 1))
y_hat = model.predict(test_input, verbose=0)[0][0]

print("y_test:", y_test)
print("y_hat:", y_hat)

model.evaluate(test_input, np.array([y_test]))

In [ ]:
# Ahora probaremos con LSTM, qué es más compleja y por lo tanto
# requiere más parámetros a entrenar
model2 = Sequential()
model2.add(LSTM(64, activation='relu', input_shape=input_shape))
model2.add(Dense(output_shape))
model2.compile(loss='mse',
              optimizer="Adam")
model2.summary()

In [ ]:
hist2 = model2.fit(X, y, epochs=500, validation_split=0.2, batch_size=5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist2.history['loss']) + 1)
sns.lineplot(x=epoch_count,  y=hist2.history['loss'], label='train')
sns.lineplot(x=epoch_count,  y=hist2.history['val_loss'], label='valid')
plt.show()

In [ ]:
# Ensayo
# x = 30
# y_test = x * 15

x_test = 30
y_test = x_test * 15
test_input = np.array([x_test])
test_input = test_input.reshape((1, 1, 1))
y_hat = model2.predict(test_input, verbose=0)[0][0]

print("y_test:", y_test)
print("y_hat:", y_hat)

model2.evaluate(test_input, np.array([y_test]))

Se puede observar que para un problema tan simple como este no hay mucha diferencia entre utilizar una RNN o LSTM.

### 3 - Multi-layer LSTM

In [ ]:
# En esta oportunidad se utilizarán dos layer LSTM. Para poder conectar
# la primera layer con la segunda se debe colocar return_sequences=True

model3 = Sequential()
model3.add(LSTM(64, activation='relu', return_sequences=True, input_shape=input_shape))
model3.add(LSTM(64, activation='relu'))
model3.add(Dense(output_shape))
model3.compile(loss='mse',
              optimizer="Adam")

model3.summary()

In [ ]:
hist3 = model3.fit(X, y, epochs=500, validation_split=0.2, batch_size=5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist3.history['loss']) + 1)
sns.lineplot(x=epoch_count,  y=hist3.history['loss'], label='train')
sns.lineplot(x=epoch_count,  y=hist3.history['val_loss'], label='valid')
plt.show()

In [ ]:
# Ensayo
# x = 30
# y_test = x * 15

x_test = 30
y_test = x_test * 15
test_input = np.array([x_test])
test_input = test_input.reshape((1, 1, 1))
y_hat = model3.predict(test_input, verbose=0)[0][0]

print("y_test:", y_test)
print("y_hat:", y_hat)

model3.evaluate(test_input, np.array([y_test]))

### 4 - Conclusión
Implementar un modelo basado en RNN o LSTM es muy sensillo, hay que tener en cuenta que al apilar varias layers hay que colocar el flag "return_sequence" en "True".
El resultado alcanzado es bueno pero podría mejorarse agregando más layer LSTM o más layer Densas